In [1]:
#to request the data from the web servers
import requests as r
#works with a parser, to extract data from HTML 
from bs4 import BeautifulSoup as bs

In [2]:
#to load the webpage content from the url
webpag = r.get("https://attack.mitre.org/groups/")

In [3]:
#convert to beautifulsoup
soup = bs(webpag.content,"html.parser")

In [4]:
#tableHead = soup.thead
#tableHead  (if u want to get the tablehead)

#if u want to extract the rowHeaders from the table head
# rowHeaders = []
# for x in tableHead.find_all('tr'):
#     for y in x.find_all('th'):
#         rowHeaders.append(y.text)
# rowHeaders

In [5]:
#CODE JUNK -  FOR MY FUTURE REF
# urlS = []
# tableBody = soup.tbody
#THIS IS HOW YOU CAN GET ALL THE LINKS FROM A Table.body
# urls_list = tableBody.find_all('a')
# print(urls_list)
# for link in urls_list:
#     href = link.get('href')
#     urls.append(href)
# urls2 = set(urls)
# urls2List = list(urls2)
# firstelem = urls2List[0]
# newUrls = []
# for elem in urls2List:
#     if elem[:6] == '/group':
#         newUrls.append(elem)
#newUrls is the finalList of my Urls.
#newUrls

### CODE STARTS FROM HERE ###
table1 = soup.find('table',{'class': "table table-bordered table-alternate mt-2"})

#if you want to find the headers in the table
# headers = []
# for i in table1.find_all('th'):
#     title = i.text
#     headers.append(title)
# headers

In [6]:
tableValues = []
for x in table1.find_all('tr')[1:]:
    #convert it to sets, to get rid of the duplicates
    #because the same name is referenced multiple times 
    td_tags = list(set(x.find_all('a')))
    td_val = [y for y in td_tags]
    tableValues.append(td_val)
#contains [[<a href="/software/S0012">PoisonIvy</a, <a href="/groups/G0018"> G0018 </a>,..] ...] for each row
tableValues

nameList = []
#from that raw html list, we extract the names i.e., texts only for each group
for elem in tableValues:
    my_val = [y.text for y in elem]
    nameList.append(list(set(my_val)))

#for each of the elem in html list, we extract its link
url_List = []
for elem in tableValues:
    my_urls = [x.get('href') for x in elem]
    url_List.append(list(set(my_urls)))

#only want names with ID's i.e., G****
clean_nameList = []
for x in nameList:
    for y in x:
        if y[1:3] ==  "G0" or y[1:3] == "G1":
            clean_nameList.append(y)

In [7]:

#only want the links for the ID's G****
clean_urlList = []
for i in range(len(url_List)):
    elem = url_List[i]
    name = clean_nameList[i][1:-1]
    for x in elem:
        if x[-5:] == name:
            clean_urlList.append(x)
            
#now zip the two lists
group_list = list(zip(clean_nameList, clean_urlList))
# print(final_list[0][1])

In [8]:
#now create the new_webpage thing using the urls collected from above:
link = "https://attack.mitre.org" + group_list[0][1] + "/"
#to load the webpage content from the url
webpag1 = r.get(link)
#convert to beautifulsoup
soup = bs(webpag1.content,"html.parser")
### CODE STARTS FROM HERE ###
table_techniques = soup.find('table',{'class': "table techniques-used background table-bordered"})
tableValues = []

for x in table_techniques.find_all('tr')[1:]:
    #we don't convert it to sets, to get rid of the duplicates
    #because in this case, we care about the order for zipping in the end
    td_tags = list(x.find_all('a'))
    td_val = [y for y in td_tags]
    tableValues.append(td_val)
#contains [[<a href="/software/S0012">PoisonIvy</a, <a href="/groups/G0018"> G0018 </a>,..] ...] for each row
tableValues

nameList = []
#from that raw html list, we extract the names i.e., texts only for each group
for elem in tableValues:
    my_val = [y.text for y in elem]
    nameList.append(list(my_val))

#for each of the elem in html list, we extract its link
url_List = []
for elem in tableValues:
    my_urls = [x.get('href') for x in elem]
    url_List.append(list(my_urls))
    

In [9]:
#zip these two:
final = list(zip(nameList, url_List))
#print(final[0])
techniques_List = []
for elem in final:
    techniques = elem[0]
    links = elem[1]
    temp  = []
    for i in range(len(links)):
        if links[i][:11] == "/techniques":
            temp.append((techniques[i], links[i]))
    techniques_List.append(temp)
            
techniques_List

[[('T1087', '/techniques/T1087'),
  ('.001', '/techniques/T1087/001'),
  ('Account Discovery', '/techniques/T1087'),
  ('Local Account', '/techniques/T1087/001')],
 [('T1059', '/techniques/T1059'),
  ('.003', '/techniques/T1059/003'),
  ('Command and Scripting Interpreter', '/techniques/T1059'),
  ('Windows Command Shell', '/techniques/T1059/003')],
 [('T1203', '/techniques/T1203'),
  ('Exploitation for Client Execution', '/techniques/T1203')],
 [('T1083', '/techniques/T1083'),
  ('File and Directory Discovery', '/techniques/T1083')],
 [('T1036', '/techniques/T1036'),
  ('.005', '/techniques/T1036/005'),
  ('Masquerading', '/techniques/T1036'),
  ('Match Legitimate Name or Location', '/techniques/T1036/005')],
 [('T1069', '/techniques/T1069'),
  ('.001', '/techniques/T1069/001'),
  ('Permission Groups Discovery', '/techniques/T1069'),
  ('Local Groups', '/techniques/T1069/001')],
 [('T1566', '/techniques/T1566'),
  ('.001', '/techniques/T1566/001'),
  ('Phishing', '/techniques/T1566'),

In [10]:
#Now do the similar thing to extract all the software used and it's url
table_software = soup.find('table',{'class': "table table-bordered table-alternate mt-2"})
tableValues = []

for x in table_software.find_all('tr')[1:]:
    #we dont convert it to sets, because we care about the order
    td_tags = list(x.find_all('a'))
    td_val = [y for y in td_tags]
    tableValues.append(td_val)
#contains [[<a href="/software/S0012">PoisonIvy</a, <a href="/groups/G0018"> G0018 </a>,..] ...] for each row
tableValues

nameList = []
#from that raw html list, we extract the names i.e., texts only for each group
for elem in tableValues:
    my_val = [y.text for y in elem]
    nameList.append(list(my_val))

#for each of the elem in html list, we extract its link
url_List = []
for elem in tableValues:
    my_urls = [x.get('href') for x in elem]
    url_List.append(list(my_urls))
#zip these two:
final = list(zip(nameList, url_List))

In [11]:
software_List = []
for elem in final:
    techniques = elem[0]
    links = elem[1]
    temp  = []
    for i in range(len(links)):
        if links[i][:9] == "/software":
            temp.append((techniques[i], links[i]))
    software_List.append(temp)
            
software_List

[[('S0043', '/software/S0043'), ('BUBBLEWRAP', '/software/S0043')],
 [('S0100', '/software/S0100'), ('ipconfig', '/software/S0100')],
 [('S0042', '/software/S0042'), ('LOWBALL', '/software/S0042')],
 [('S0039', '/software/S0039'), ('Net', '/software/S0039')],
 [('S0104', '/software/S0104'), ('netstat', '/software/S0104')],
 [('S0012', '/software/S0012'), ('PoisonIvy', '/software/S0012')],
 [('S0096', '/software/S0096'), ('Systeminfo', '/software/S0096')]]